Code based of the implementation at: https://github.com/papabloblo/semi_boost

In [1]:
import numpy as np
from sklearn import neighbors
from sklearn.svm import SVC
from scipy import sparse
from scipy.spatial.distance import pdist,squareform
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import time
import math

class SemiBoostClassifier():

    def __init__(self, base_model = SVC(probability=True)):
        self.BaseModel = base_model

    '''Fit model'''
    def fit(self, X, y1,
            n_neighbors=4, n_jobs = 1,
            max_models = 15,
            sample_percent = 0.5,
            sigma_percentile = 90,
            similarity_kernel = 'rbf',
            verbose = False):
        
        # Convert from Dataframe to npArray
        y = np.array(y1)
        
        # Localize labeled data
        #idx_label = np.argwhere((y == labels[0]) | (y == labels[1])).flatten()
        #idx_not_label = np.argwhere(y == unlabels[0]).flatten()
        l = []
        u = []
        for i in range(y.size):
            if y[i] in [0,1]:
                l.append(i)
            else:
                u.append(i)
        idx_label = np.array(l)
        idx_not_label = np.array(u)

        # The parameter C is defined in the paper as
        if idx_not_label.shape[0]!=0:
            C = idx_label.shape[0]/idx_not_label.shape[0]
        else:
            C = float('inf')

        # First we need to create the similarity matrix
        if similarity_kernel == 'knn':

            self.S = neighbors.kneighbors_graph(X,
                                                n_neighbors=n_neighbors,
                                                mode='distance',
                                                include_self=True,
                                                n_jobs=n_jobs)

            self.S = sparse.csr_matrix(self.S)

        elif similarity_kernel == 'rbf':
            # First aprox
            self.S = np.sqrt(rbf_kernel(X, gamma = 1))
            # set gamma parameter as the 15th percentile
            sigma = np.percentile(np.log(self.S), sigma_percentile)
            sigma_2 = (1/sigma**2)*np.ones((self.S.shape[0],self.S.shape[0]))
            self.S = np.power(self.S, sigma_2)
            # Matrix to sparse
            self.S = sparse.csr_matrix(self.S)

        else:
            print('No kernel type ', similarity_kernel)
            
        if idx_not_label.shape[0]==0:
            self.models = []
            clf = self.BaseModel
            clf.fit(X, y)
            self.models.append(clf)
            return

        #=============================================================
        # Initialise variables
        #=============================================================
        self.models = []
        self.weights = []
        H = np.zeros(idx_not_label.shape[0])

        # Loop for adding sequential models
        for t in range(max_models):
            
            #=============================================================
            # Calculate p_i and q_i for every sample
            #=============================================================
            p_1 = np.einsum('ij,j', self.S[:,idx_label].todense(), (y[idx_label]==1))[idx_not_label]*np.exp(-2*H)
            p_2 = np.einsum('ij,j', self.S[:,idx_not_label].todense(), np.exp(H))[idx_not_label]*np.exp(-H)
            p = np.add(p_1, p_2)
            p = np.squeeze(np.asarray(p))

            q_1 = np.einsum('ij,j', self.S[:,idx_label].todense(), (y[idx_label]==-1))[idx_not_label]*np.exp(2*H)
            q_2 = np.einsum('ij,j', self.S[:,idx_not_label].todense(), np.exp(-H))[idx_not_label]*np.exp(H)
            q = np.add(q_1, q_2)
            q = np.squeeze(np.asarray(q))

            #=============================================================
            # Compute predicted label z_i
            #=============================================================
            z = np.sign(p-q)
            z_conf = np.abs(p-q)
            
            #=============================================================
            # Sample sample_percent most confident predictions
            #=============================================================
            # Sampling weights
            sample_weights = z_conf/np.sum(z_conf)
            
            # If there are non-zero weights
            if np.any((sample_weights != 0)):
                if z.size != 1 :
                    idx_aux = np.random.choice(np.arange(z.size), size = int(sample_percent*idx_not_label.size), p = sample_weights, replace = False)
                else :
                    idx_aux = [0]
                    
                idx_sample = idx_not_label[idx_aux]
            else:
                print('No similar unlabeled observations left.')
                break

            # Create new X_t, y_t
            idx_total_sample = np.concatenate([idx_label,idx_sample]).flatten().tolist()
            #X_t = X[idx_total_sample,]
            X_t = X.iloc[idx_total_sample]
            if z.size !=1:
                np.put(y, idx_sample, z[idx_aux]) # Include predicted to train new model
            else:
                #print("y[idx_sample]: "+ str(y[idx_sample]))
                y[idx_sample] = z
            y_t = y[idx_total_sample]

            #=============================================================
            # Fit BaseModel to samples using predicted labels
            #=============================================================
            # Fit model to unlabeled observations
            clf = self.BaseModel
            clf.fit(X_t, y_t)
            # Make predictions for unlabeled observations
            h = clf.predict(X.iloc[idx_not_label])

            # Refresh indexes
            idx_label = idx_total_sample
            idx_not_label = np.array([i for i in np.arange(y.size) if i not in idx_label])
                
            # If no samples are left without label, break
            if idx_not_label.size == 0:
                if verbose:
                    print('All observations have been labeled')
                    print('Number of iterations: ',t + 1)
                break
            else:
                if verbose:
                    print('There are still ', idx_not_label.shape[0], ' unlabeled observations')

            #=============================================================
            # Compute weight (a) for the BaseModel as in (12)
            #=============================================================
            e = (np.dot(p,h==-1) + np.dot(q,h==1))/(np.sum(np.add(p,q)))
            a = 0.25*np.log((1-e)/e)
            
            #=============================================================
            # Update final model
            #=============================================================
            # If a<0 the model is not converging
            if a<0:
                if verbose:
                    print('Problematic convergence of the model. a<0')
                break

            # Save model
            self.models.append(clf)
            #save weights
            self.weights.append(a)
            # Update
            H = np.zeros(idx_not_label.size)
            w = np.sum(self.weights)
            for i in range(len(self.models)):
                H = np.add(H, self.weights[i]*self.models[i].predict(X.iloc[idx_not_label]))
                # H = np.add(H, self.weights[i]*self.models[i].predict_proba(X[idx_not_label])[:,1]/w)

            # H = np.array(list(map(lambda x: 1 if x>0 else -1, H)))
            #=============================================================
            # Breaking conditions
            #=============================================================

            # Maximum number of models reached
            if (t==max_models) & verbose:
                print('Maximum number of models reached')

            # If no samples are left without label, break
            if idx_not_label.size == 0:
                if verbose:
                    print('All observations have been labeled')
                    print('Number of iterations: ',t + 1)
                break

        if verbose:
            print('\n The model weights are \n')
            print(self.weights)

    def predict(self, X, semi):
        if semi:
            estimate = np.zeros(X.shape[0])
            # Predict weighting each model
            w = np.sum(self.weights)
            for i in range(len(self.models)):
                estimate = np.add(estimate,  self.weights[i]*self.models[i].predict_proba(X)[:,1]/w)
                # estimate = np.add(estimate, self.weights[i]*self.models[i].predict(X))
            estimate = np.array(list(map(lambda x: 1 if x>0 else -1, estimate)))
            estimate = estimate.astype(int)
            return estimate
        else:
            return self.models[-1].predict(X)
    
    def predict_proba(self, X, semi):
        if semi:
            estimate = np.zeros(X.shape[0])
            # Predict weighting each model
            w = np.sum(self.weights)
            for i in range(len(self.models)):
                estimate = np.add(estimate,  self.weights[i]*self.models[i].predict_proba(X)[:,1]/w)
                # estimate = np.add(estimate, self.weights[i]*self.models[i].predict(X))
            # estimate = np.array(list(map(lambda x: 1 if x>0 else -1, estimate)))
            # estimate = estimate.astype(int)
            return estimate
        else:
            return self.models[-1].predict_proba(X)[:,1]


# Testing the Code

In [2]:
filename2 = 'marketing_campaign'

df2r0 = pd.read_csv("../data/train/noresampling/"+filename2+"_0.csv", index_col=0)
df2r10 = pd.read_csv("../data/train/noresampling/"+filename2+"_10.csv", index_col=0)
df2r20 = pd.read_csv("../data/train/noresampling/"+filename2+"_20.csv", index_col=0)
df2r50 = pd.read_csv("../data/train/noresampling/"+filename2+"_50.csv", index_col=0)
df2r90 = pd.read_csv("../data/train/noresampling/"+filename2+"_90.csv", index_col=0)
df2r95 = pd.read_csv("../data/train/noresampling/"+filename2+"_95.csv", index_col=0)

df2t = pd.read_csv("../data/test/"+filename2+".csv", index_col=0)

numerical_features2 = ['Income','MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts','MntSweetProducts', 
                        'MntGoldProds','Year_Birth','Recency','NumDealsPurchases','NumWebPurchases',
                        'NumCatalogPurchases','NumStorePurchases','NumWebVisitsMonth','Dt_Customer']
categorical_features2 = ['Education','Marital_Status','Kidhome','AcceptedCmp3', 
                        'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2','Complain','Response']
target2 = 'Teenhome'

In [3]:
def SemiBoost_Test(dftrain, dftest, categorical_features, numerical_features, target, model, semi):
        
    start_time = time.time()
    
    X_train = dftrain[numerical_features+categorical_features]
    y_train = dftrain[target]
    pseudo = y_train.fillna(-1)
    
    X_test = dftest[numerical_features+categorical_features]
    y_test = dftest[target]
    
    #print(pseudo.to_list())
    
    X_train[categorical_features] = X_train[categorical_features].astype('category')
    X_test[categorical_features] = X_test[categorical_features].astype('category')
    
    model.fit(X_train, pseudo, verbose=True)
    y_pred = model.predict(X_test, semi)
    y_pred_prob = model.predict_proba(X_test, semi)
    
    execution = time.time() - start_time
    
    return [execution, y_pred, y_pred_prob]

In [4]:
#Sanity Check

#[t, p, pp] = SemiBoost_Test(df2r10, df2t, categorical_features2, numerical_features2, target2, SemiBoostClassifier(), False)

#pp

c:\users\alexa\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


There are still  78  unlabeled observations
There are still  39  unlabeled observations
There are still  20  unlabeled observations
There are still  10  unlabeled observations
There are still  5  unlabeled observations
There are still  3  unlabeled observations
There are still  2  unlabeled observations
There are still  1  unlabeled observations
All observations have been labeled
Number of iterations:  9

 The model weights are 

[0.5802541167664359, 0.5392112052236815, 0.44367837029138363, 0.2558947023181577, 0.2780124979226367, 0.28699693165665857, 0.266309732728326, 0.2379299115058545]


array([0.51530761, 0.54054618, 0.55251369, 0.52326889, 0.53263162,
       0.53119059, 0.55065673, 0.54546814, 0.51671239, 0.50664431,
       0.54331925, 0.51961713, 0.53100344, 0.59038943, 0.51046894,
       0.52892112, 0.5199755 , 0.51156305, 0.53021107, 0.50692842,
       0.52658174, 0.53150533, 0.53159059, 0.52675501, 0.52750565,
       0.53130232, 0.54950391, 0.54826672, 0.55697558, 0.5287919 ,
       0.53339935, 0.53449491, 0.53634484, 0.52710062, 0.53477151,
       0.54826289, 0.5       , 0.53686077, 0.52909684, 0.52834037,
       0.52053795, 0.51479064, 0.53717985, 0.51998995, 0.54964899,
       0.53260119, 0.58504457, 0.54870365, 0.51899326, 0.53853956,
       0.52889972, 0.57022844, 0.53371692, 0.56497019, 0.53507553,
       0.5143968 , 0.51753181, 0.55001352, 0.53407457, 0.52277026,
       0.57155271, 0.52147576, 0.52575933, 0.52793597, 0.56723799,
       0.52226391, 0.5784903 , 0.53099052, 0.51670347, 0.53266294,
       0.53507615, 0.58440743, 0.53194054, 0.53014051, 0.55143

In [5]:
#print(p)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.